# [실습] Advanced RAG

RAG의 기본 베이스 체인에서 시작하여, 다양한 기능을 추가해 보겠습니다.   

### <필수> 실습을 진행하기 전, GPU를 T4로 설정해 주세요!

### 라이브러리 설치  

랭체인 관련 라이브러리와 벡터 데이터베이스 라이브러리를 설치합니다.   

In [ ]:
!pip install jsonlines openai langchain langchain-openai langchain-community langchain-chroma tiktoken rank_bm25 pymupdf kiwipiepy

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-chroma to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 35.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

### LLM과 임베딩 모델 불러오기

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

load_dotenv('env', override=True)
if os.environ.get('OPENAI_API_KEY'):
    print('OpenAI API 키 확인')

llm = ChatOpenAI(model="gpt-4.1-mini", temperature = 0.7)
sllm = ChatOpenAI(model='gpt-4.1-nano', temperature = 0.7)
reasoning_llm = ChatOpenAI(model='gpt-5-mini')

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')

OpenAI API 키 확인


pdfs.zip 파일을 압축 해제합니다.

In [ ]:
import zipfile

with zipfile.ZipFile('pdfs.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

## 데이터 불러오기

폴더에 포함된 문서들을 pdf 로더로 불러옵니다.   
glob 라이브러리를 사용합니다.

In [ ]:
from langchain.schema import Document
from glob import glob

# 모든 PDF 파일을 glob으로 찾음
pdf_files = glob("./*.pdf")
pdf_files

['./분기보고서(2024.11.14).pdf',
 './사업보고서(2025.03.11).pdf',
 './반기보고서(2024.08.14).pdf']

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

# 각 PDF 파일에서 페이지별로 내용을 불러와 하나로 합침
all_papers=[]

for i, path_paper in enumerate(pdf_files):
    loader = PyMuPDFLoader(path_paper)
    pages = loader.load()
    doc = Document(page_content='', metadata = {'index':i, 'source':pages[0].metadata['source']})
    for page in pages:
        doc.page_content += page.page_content +' '

    doc.page_content = doc.page_content.replace('\n', ' ')
    for _ in range(10):
        doc.page_content = doc.page_content.replace('  ', ' ')
        doc.page_content = doc.page_content.replace('..', '.')

    all_papers.append(doc)

print(len(all_papers))
all_papers[0].page_content[:1000]

3


'목 차 분 기 보 고 서.1 【 대표이사 등의 확인 】.2 I. 회사의 개요.3 1. 회사의 개요.3 2. 회사의 연혁.3 3. 자본금 변동사항.3 4. 주식의 총수 등.3 5. 정관에 관한 사항.3 II. 사업의 내용.4 1. 사업의 개요.4 2. 주요 제품 및 서비스.5 3. 원재료 및 생산설비.9 4. 매출 및 수주상황.11 5. 위험관리 및 파생거래.13 6. 주요계약 및 연구개발활동.15 7. 기타 참고사항.17 III. 재무에 관한 사항.26 1. 요약재무정보.26 2. 연결재무제표.29 2-1. 연결 재무상태표.29 2-2. 연결 손익계산서.30 2-3. 연결 포괄손익계산서.31 2-4. 연결 자본변동표.31 2-5. 연결 현금흐름표.32 3. 연결재무제표 주석.34 1. 지배기업의 개요 (연결) .34 2. 연결재무제표 작성기준 및 중요한 회계정책 (연결).34 3. 중요한 판단과 추정 불확실성의 주요 원천 (연결) .35 4. 영업부문 (연결) .36 5. 범주별 금융상품 (연결) .39 6. 공정가치 (연결) .40 7. 공정가치측정금융자산 (연결).44 8. 관계기업투자주식 (연결).47 9. 종속기업 (연결) .49 10. 유형자산 (연결) .57 11. 무형자산 (연결) .57 12. 투자부동산 (연결).59 13. 리스 (연결) .60 14. 리스부채 (연결) .62 15. 금융리스채권 (연결).64 16. 충당부채 (연결) .65 17. 재무위험관리 (연결).68 18. 우발채무와 약정사항 (연결) .70 19. 납입자본 (연결) .73 20. 이익잉여금 (연결).73 21. 기타자본항목 (연결).74 22. 매출액 (연결).75 23. 판매비와 관리비 (연결).78 24. 기타수익 및 기타비용 (연결).80 25. 금융수익 및 금융비용 (연결).81 26. 법인세비용 (연결).83 27. 현금흐름표 (연결).83 28. 주당이익 (연결) .86 29. 특수관계자 (연결).87 30. 주식기준보상 (연결).96 4. 재무제표.100

글자 수와 토큰 수를 확인해 보겠습니다.

In [ ]:
import tiktoken

encoder = tiktoken.encoding_for_model('gpt-4o-mini')
for paper in all_papers:
    print(len(paper.page_content), len(encoder.encode(paper.page_content)), paper.metadata['source'])

212976 123712 ./분기보고서(2024.11.14).pdf
510249 283113 ./사업보고서(2025.03.11).pdf
345728 182859 ./반기보고서(2024.08.14).pdf


# 토큰 단위로 청킹하기   

tiktoken이나 huggingface를 이용해 토큰 단위 청킹을 수행합니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken
token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",
    chunk_size=2000,
    chunk_overlap=400,
)
# from_huggingface_tokenizer : 허깅페이스 모델에서 토크나이저 가져오기

token_chunks = token_splitter.split_documents(all_papers)
print(len(token_chunks))

371


벡터 DB를 구성합니다.   

In [ ]:
from langchain_chroma import Chroma
from tqdm import tqdm

Chroma().delete_collection()
db = Chroma(embedding_function=embeddings,
                           persist_directory="./chroma",
                           collection_metadata={'hnsw:space':'l2'},
                           collection_name='finance',
                           )


# 50개씩 저장
for i in tqdm(range(0, len(token_chunks), 50)):
    db.add_documents(token_chunks[i:min(i+50, len(token_chunks))])

retriever = db.as_retriever(search_kwargs={"k": 5})

# filter 옵션을 통해 특정 메타데이터를 가진 벡터만 검색 가능
# retriever = db.as_retriever(search_kwargs={"k": 5,"filter":{'author':'Hyungho Byun'}})

100%|██████████| 8/8 [00:27<00:00,  3.41s/it]


RAG 체인을 구현합니다.

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ("user", '''당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 한국어로 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])
prompt.pretty_print()

================================ Human Message =================================

당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 한국어로 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

def format_docs(docs):
    return "\n---\n".join([doc.page_content for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}

    | prompt
    | llm
    | StrOutputParser()
)

다양한 질문을 통해 결과를 확인해 보겠습니다.

In [ ]:
questions = [
    '이 회사의 사업은 크게 두 부문으로 나눠집니다. 각각은 어떤 부문입니까?',
    'IT서비스는 몇 개의 분야로 나눠집니까? 각각은 무엇입니까?',
    '2024년과 2025년에 걸쳐, 매출 비중은 어떻게 변화했나요?',
    '2024년 6월말 기준, 연결 재무제표의 유동자산은 얼마인가요?',
    'GPUaaS 서비스가 무엇입니까?',
    '회사의 연혁 중 2024년 3월 20일 주주총회에서 이인실 사외이사가 신규 선임되었는데, 이때 함께 재선임된 사내이사는 누구이며 사임한 사외이사는 누구입니까?',
    '회사의 연구개발 담당조직은 어떤 분야의 핵심 기술을 연구하고 있습니까?',
    '2021-2022년 사이에 있었던 합병 사례를 모두 파악하여, 표로 나타내세요.'
]
result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')


Question: 이 회사의 사업은 크게 두 부문으로 나눠집니다. 각각은 어떤 부문입니까?
Answer: 이 회사의 사업은 크게 두 부문으로 나뉘며, 각각 IT서비스 부문과 물류 부문입니다.
---
Question: IT서비스는 몇 개의 분야로 나눠집니까? 각각은 무엇입니까?
Answer: IT서비스는 3개의 분야로 나눠집니다. 각각은 다음과 같습니다.

1. 클라우드 서비스  
2. SI(시스템 통합)  
3. ITO(IT 아웃소싱)
---
Question: 2024년과 2025년에 걸쳐, 매출 비중은 어떻게 변화했나요?
Answer: 2024년과 2025년에 걸쳐 매출 비중 변화에 대한 구체적인 수치는 제공된 Context에 직접적으로 명시되어 있지 않습니다.

다만, 2024년 사업부문별 매출 비중은 다음과 같습니다.  
- IT서비스 사업부문 매출 비중: 약 46.3% (6조 4,014억원)  
- 물류 사업부문 매출 비중: 약 53.7% (7조 4,268억원)

2024년 IT서비스 매출 내 클라우드 사업의 비중이 2023년 31%에서 2024년 36%로 증가한 점은 언급되어 있습니다.

2025년 전망 관련해서는 글로벌 IT서비스 시장과 국내 IT서비스 시장의 성장률이 각각 9.0%, 9.9%로 높게 전망되고 있으며, 물류 시장도 3PL 물류 시장 연평균 5.6%, 디지털포워딩 시장 연평균 18.9% 성장할 것으로 예측되어 IT서비스 부문의 성장 비중이 커질 가능성이 있으나, 구체적인 매출 비중 수치는 제공되지 않았습니다.

따라서, 2024년과 2025년 매출 비중의 구체적인 변화 수치는 정보가 부족하여 정확히 답변할 수 없습니다.
---
Question: 2024년 6월말 기준, 연결 재무제표의 유동자산은 얼마인가요?
Answer: 2024년 6월말 기준, 연결 재무제표의 유동자산은 8,436,908,715,388원입니다.
---
Question: GPUaaS 서비스가 무엇입니까?
Answer: GPUaaS(GPU as a Service)는 생성

# Multi-Query Retriever   
모호한 쿼리를 검색하는 대신, 다양한 관점에서 Paraphrazing한 쿼리를 사용할 수 있습니다.   
이 때, LLM의 도움을 받을 수 있습니다.

In [ ]:
# Multi Query를 확인하기 위한 로깅
import logging

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever

rewrite_prompt = PromptTemplate(template = """
당신은 삼성SDS 임직원을 대상으로 하는 정보 검색 봇입니다.

기업 공시 문서에 대한 질문이 주어집니다.
해당 질문에 대한 정보를 검색하기 위해, 벡터 데이터베이스에 입력할 다양한 맥락의 질문을 생성하세요.
질문은 3개 생성하고, 한 줄에 질문 하나씩 출력하세요.
질문을 다각도로 분석하여, 다양한 검색 결과가 나오도록 구성해야 합니다.

공시 문서에서는 삼성에스디에스, Samsung SDS 두 표현이 혼재되어 있으므로 다양한 질문을 생성하세요.

---
원본 질문: {question}

""")

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(),
    llm=llm,
    prompt = rewrite_prompt,
)

# prompt + llm 으로 질문을 3개로 확장, 각각 retriver 로 3*k개 검색, 합집합으로 압축

In [ ]:
len(multi_query_retriever.invoke("우리 회사는 어떤 사업이 제일 매출 비중이 높니?"))

INFO:langchain.retrievers.multi_query:Generated queries: ['삼성SDS의 주요 사업별 매출 비중은 어떻게 되나요?  ', 'Samsung SDS 사업 부문별 매출 비중 현황을 알려주세요.  ', '삼성에스디에스의 각 사업 분야별 매출 점유율과 순위는 무엇인가요?']


10

In [ ]:
rag_chain = (
    {"context": multi_query_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')


INFO:langchain.retrievers.multi_query:Generated queries: ['GPUaaS 서비스의 정의와 주요 기능은 무엇인가요?  ', 'GPUaaS가 제공하는 혜택과 활용 사례에는 어떤 것들이 있나요?  ', 'GPUaaS 서비스와 전통적인 GPU 사용 방식의 차이점은 무엇인가요?']
INFO:langchain.retrievers.multi_query:Generated queries: ['이 회사가 영위하는 주요 사업 부문은 무엇입니까?  ', '회사의 사업 부문별 구성은 어떻게 되어 있습니까?  ', '기업의 사업 영역을 구분하는 기준과 각 부문의 명칭은 무엇인가요?']
INFO:langchain.retrievers.multi_query:Generated queries: ['2024년과 2025년의 매출 비중 변화 추이를 알려주세요.  ', '2024년 대비 2025년 매출 구성 비율에 어떤 차이가 있나요?  ', '2024년과 2025년 동안 주요 사업 부문별 매출 비중은 어떻게 변했나요?']
INFO:langchain.retrievers.multi_query:Generated queries: ['IT서비스의 세부 분야는 몇 가지이며 각각의 명칭은 무엇인가요?  ', '기업 공시 문서에서 IT서비스가 분류된 분야의 개수와 구체적인 내용을 알려주세요.  ', 'IT서비스 분야 구분과 각 분야별 주요 업무 또는 역할에 대해 설명해 주세요.']
INFO:langchain.retrievers.multi_query:Generated queries: ['2024년 6월 30일 기준 연결 재무제표에 기재된 유동자산 총액은 얼마입니까?  ', '2024년 2분기 말 연결 재무상태표 상 유동자산 금액을 알려주세요.  ', '2024년 6월 말 현재 회사의 연결 재무제표에 보고된 유동자산 규모는 어떻게 됩니까?']
INFO:langchain.retrievers.multi_query:Generated queries: ['2024년 3월 20일 

Question: 이 회사의 사업은 크게 두 부문으로 나눠집니다. 각각은 어떤 부문입니까?
Answer: 이 회사의 사업은 크게 두 부문으로 나눠지며, 각각은 다음과 같습니다.

1. IT서비스 부문  
2. 물류 부문

IT서비스 부문은 IT기술역량을 활용하여 클라우드 서비스, 시스템 통합(SI), IT 아웃소싱(ITO) 등 다양한 IT서비스를 제공하는 분야이고,  
물류 부문은 물류 통합관리 플랫폼과 디지털 물류 플랫폼을 기반으로 국제운송, 내륙운송, 물류센터 운영, 이커머스, 프로젝트 물류 등 종합 물류 서비스를 제공하는 분야입니다.
---
Question: IT서비스는 몇 개의 분야로 나눠집니까? 각각은 무엇입니까?
Answer: IT서비스는 3개의 분야로 나눠집니다. 각각은 다음과 같습니다.

1. 클라우드 서비스 (Cloud Service)  
2. SI (System Integration)  
3. ITO (IT Outsourcing)  

이 세 분야를 통해 고객 맞춤형 IT인프라, 업무시스템 구축, IT 운영 및 아웃소싱 서비스를 제공합니다.
---
Question: 2024년과 2025년에 걸쳐, 매출 비중은 어떻게 변화했나요?
Answer: 2024년 매출 비중은 IT서비스 사업부문이 46.3%, 물류 사업부문이 53.7%를 차지하였습니다. IT서비스 매출은 전년 대비 4.8% 증가하여 6조 4,014억원을 기록했으며, 물류 매출은 전년 대비 3.6% 증가한 7조 4,268억원입니다. 

2025년 매출 비중에 대한 구체적인 수치는 제시되어 있지 않으나, IT서비스 시장은 2025년에 국내 9.9%, 글로벌 9.0% 성장률이 전망되고 있으며, 클라우드 및 생성형 AI 서비스가 계속 확대될 것으로 예상되어 IT서비스 매출 비중이 증가할 가능성이 큽니다. 물류 부문은 2025년 신규 선박 투입으로 공급이 증가하고 물류 운임 하락이 예상되나, 디지털포워딩 사업의 글로벌 확대 및 계약물류 사업 확장으로 매출 성장이 기대됩니다.

따라서 2024년 


### Ensemble Retriever

Ensemble Retriever는 서로 다른 리트리버를 결합하여 순위를 합산합니다.   
주로 Keyword Indexing 기반 검색인 BM25 검색과 Semantic 검색을 합친 Hybrid Search를 사용합니다.

In [ ]:
!pip install kiwipiepy

In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(token_chunks, preprocess_func = kiwi_tokenize)
bm25_retriever.k = 5
# BM25: 키워드 기반 인덱싱 리트리버 --> kiwi_tokenize로 전처리


retriever = db.as_retriever(search_kwargs={"k": 5})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [ ]:
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')

Question: 이 회사의 사업은 크게 두 부문으로 나눠집니다. 각각은 어떤 부문입니까?
Answer: 이 회사의 사업은 크게 두 부문으로 나누어지며, 각각은 다음과 같습니다.

1. IT서비스 부문  
- 클라우드 서비스(CSP, MSP, SaaS)  
- 시스템 통합(SI)  
- IT 아웃소싱(ITO)  
이 부문에서는 고객 맞춤형 클라우드 인프라 서비스, 클라우드 도입 컨설팅 및 운영, 엔터프라이즈 업무 솔루션, ERP, SCM, CRM 등 업무 시스템 구축과 관련 컨설팅, IT 아웃소싱 서비스 등을 제공합니다.

2. 물류 부문  
- 글로벌 종합 물류 서비스  
- 자체 개발한 물류 통합관리 플랫폼 Cello와 디지털 물류 플랫폼 Cello Square를 통한 국제운송, 내륙운송, 물류센터 운영, 프로젝트 물류 등 전 영역 서비스  
- 디지털포워딩 서비스 제공  
이 부문은 전 세계 다수 국가에 걸친 글로벌 네트워크와 IT기술을 활용하여 차별화된 물류 솔루션과 종합 물류 서비스를 제공합니다.
---
Question: IT서비스는 몇 개의 분야로 나눠집니까? 각각은 무엇입니까?
Answer: IT서비스는 3개의 분야로 나눠집니다. 각각은 클라우드 서비스, SI, ITO입니다.
---
Question: 2024년과 2025년에 걸쳐, 매출 비중은 어떻게 변화했나요?
Answer: 2024년과 2025년에 걸쳐 매출 비중 변화는 다음과 같습니다.

- 2024년 IT서비스 사업부문 매출은 6조 4,014억원으로 전체 매출의 약 46.3%를 차지하였고, 물류 사업부문 매출은 7조 4,268억원으로 약 53.7%를 차지하였습니다.

- 2025년 전망 자료에 따르면, 국내 IT서비스 시장 규모는 33.2조원으로 전년 대비 9.9% 증가가 예상되며, 연평균 10.1% 성장하여 2028년에는 44.3조원 규모에 이를 전망입니다.

- 물류 부문은 2025년부터 2029년까지 글로벌 3PL 물류 시장이 연평균 5.6% 성장할 것으로 전망되며, 디지털포워딩 시장

# Reranker
넓은 Retriever 검색 범위를 이용한 뒤, Reranker를 통해 개수를 줄일 수 있습니다.

In [ ]:
rough_retriever = db.as_retriever(search_kwargs={"k": 20})

In [ ]:
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers import ContextualCompressionRetriever
import torch

# 다국어 리랭커 모델 사용하기
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

compressor = CrossEncoderReranker(model=model, top_n=5)
# Cross Encoder : 질문과 Chunk를 같이 넣고 처리하는 Transformers 계열 모델
# 점수순으로 Top 5

compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=rough_retriever)
compressed_docs = compression_retriever.invoke("2021-2022년 사이에 있었던 합병 사례를 모두 파악하여, 표로 나타내세요.")

compressed_docs

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

[Document(id='24ff2c53-f953-4578-a072-7c84e4dfbad7', metadata={'source': './반기보고서(2024.08.14).pdf', 'index': 2}, page_content="최근 3년간 최근 5년간 - 29 1.86 1.63 [지분증권의 발행 등과 관련된 사항] (기준일 : 2024년 06월 30일 ) (단위 : 원, 주) 주식발행 (감소)일자 발행(감소) 형태 발행(감소)한 주식의 내용 종류 수량 주당 주당발행 비고 전자공시시스템 dart.fss.or.kr Page 176 ※ 당사는 공시대상기간 중 자본금 변동사항이 없습니다. 나. 전환사채 등 발행현황 미상환 전환사채 발행현황 ※ 해당사항 없음 미상환 신주인수권부사채 등 발행현황 ※ 해당사항 없음 미상환 전환형 조건부자본증권 발행현황 ※ 해당사항 없음 다. 채무증권 발행실적 등 액면가액 (감소)가액 - - - - - - - (기준일 : 2024년 06월 30일 ) (단위 : 원, 주) 종류＼구분 회차 발행일 만기일 권면 (전자 등록 )총액 전환대상 주식의 종 류 전환청 구가능 기간 전환조건 미상환사채 비고 전환 비율 (%) 전환가 액 권면 (전자 등록 )총액 전환가 능주식 수 - - - - - - - - - - - - 합 계 - - - - - - - - - - - (기준일 : 2024년 06월 30일 ) (단위 : 원, 주) 종류＼구분 회차 발행일 만기일 권면 (전자 등록 )총액 행사대상 주식의 종 류 신주인 수권 행 사가능 기간 행사조건 미상환 사채 미행사 신주 인수권 비고 행사비 율 (%) 행사가 액 권면(전 자등록 )총액 행사가 능주식 수 - - - - - - - - - - - - 합 계 - - - - - - - - - - - (기준일 : 2024년 06월 30일 ) (단위 : 원, 주) 구분 회차 발행일 만기일 권면 (전자 등록 )총액 주식 전환의 사유 주식 전환의 범위 전환가 액 주식전환으로 인하여 발행할 주식 비고 종류 수 - - - - - -

# Contextual Retrieval    

Claude가 제안한 Contextual Retrieval은 전체 Context를 활용하여     
청크별 헤더를 추가하는 방법입니다.    

In [ ]:
# 청크 초기화
token_chunks = token_splitter.split_documents(all_papers)
print(len(token_chunks))

371


In [ ]:
context_prompt = ChatPromptTemplate(
    [
        ('user', '''
기업 공시 보고서의 전체 내용과, 그 중 일부 Chunk가 주어집니다.
주어진 Document의 일부인 Chunk에 대해
간결하고 관련성 있는 짧은 설명을 생성하세요.
청크만으로는 명확하지 않으나, 전체를 참고하여 파악할 수 있는 정보를 추가하여
청크의 내용이 더 명확해지도록 하는 2~4문장 길이의 Context를 생성하면 됩니다.
아래의 가이드라인을 참고하세요.

1. 텍스트 부분에서 논의된 주요 주제나 개념을 포함하세요.
2. 문서 전체의 문맥에서 관련 정보나 비교를 언급하세요.
3. 가능한 경우, 이 정보가 문서의 전체적인 주제나 목적과 어떻게 연관되는지를 설명하세요.
4. 중요한 정보를 제공하는 주요 항목과 수치를 포함하세요.
5. 답변은 한국어로 작성합니다.

답변은 간결하게 작성하세요.

# Input Format

- [Document]: `<document> {document} </document>`
- [Chunk]: `<chunk> {chunk} </chunk>`

Context:
        ''')
    ]
)

context_chain = context_prompt | llm | StrOutputParser()


Context가 잘 생성됐는지 확인해 봅니다.

In [ ]:
chunk = token_chunks[40]
doc = all_papers[chunk.metadata['index']].page_content
context = context_chain.invoke({'document':doc, 'chunk':chunk.page_content})
print(context)
print('========')
print(chunk.page_content)

해당 청크는 주요 경영진의 급여 내역과 (주)엠로 임직원 대상 주식매입선택권 현황을 포함하고 있습니다. 경영진은 회사의 계획·운영·통제에 중대한 책임을 지며, 주식매입선택권은 2차부터 5차까지 부여되어 있으며, 총 633,540주의 선택권이 유효합니다. 이 정보는 임원 보상 및 직원 동기 부여 체계와 연계되어 당사의 인재 확보 및 유지 전략을 보여줍니다.
587,668 1,710,629 퇴직급여 124,353 375,821 합계 1,658,581 4,883,226 전분기 (단위 : 천원) 　 장부금액 　 공시금액 　 3개월 누적 단기종업원급여 986,156 2,704,797 장기급여 (1,394,893) (2,264,480) 퇴직급여 100,829 312,737 합계 (307,908) 753,054 상기의 주요 경영진은 연결실체 활동의 계획ㆍ운영ㆍ통제에 대한 중요한 권한과 책임을 가진 지배 기업의 등기이사입니다. (1) 연결실체인 (주)엠로가 임직원에 부여한 주식매입선택권 현황은 다음과 같습니다. 임직원에 부여한 주식매입선택권 현황 　 　 　 전체 종속기업 전자공시시스템 dart.fss.or.kr Page 96 종속기업 　 (주)엠로 　 주식 　 기명식 보통주식 2차 기명식 보통주식 3차 기명식 보통주식 4차 기명식 보통주식 5차 부여방법, 주식매 입선택권 이사회가 신주교부이사회가 신주교부이사회가 신주교부이사회가 신주교부 행사가능 조건, 주 식매입선택권 주식선택권 행사일 현재 재직중인 자 (사망, 정년퇴직, 임원으로의 승진 및 임원의 임기만 료 등 기타 본인의 귀책사유가 아닌 사유는 제외) 주식선택권 행사일 현재 재직중인 자 (사망, 정년퇴직, 임원으로의 승진 및 임원의 임기만 료 등 기타 본인의 귀책사유가 아닌 사유는 제외) 주식선택권 행사일 현재 재직중인 자 (사망, 정년퇴직, 임원으로의 승진 및 임원의 임기만 료 등 기타 본인의 귀책사유가 아닌 사유는 제외) 주식선택권 행사일 현재 재직중인 자 (사망, 정년퇴직, 임원으로의 승진 및 임원의 임기만 료 등 

이제 Context 추가 작업을 수행합니다.

In [ ]:
from tqdm import tqdm

chunk_with_parents=[]

for i, chunk in enumerate(tqdm(token_chunks)):
    doc = all_papers[chunk.metadata['index']].page_content
    chunk_with_parents.append({'document':doc, 'chunk':chunk.page_content})
    # print('\n'+context)
    # print('---')

# 10개씩 실행 (API 여유시 배치 늘리기)
for i in tqdm(range(0, len(token_chunks), 10)):
    contexts = context_chain.batch(chunk_with_parents[i:min(i+10, len(token_chunks))])
    for j in range(i,min(i+10, len(token_chunks))):
        token_chunks[j].page_content = context + '\n\n' + token_chunks[j].page_content

수정된 청크를 이용해, 벡터 데이터베이스를 다시 구성합니다.

In [ ]:
db = Chroma(embedding_function=embeddings,
                           persist_directory="./chroma_Web_ContextualRetrieval",
                           collection_metadata={'hnsw:space':'l2'},
                           collection_name='Report',
                           )

# 50개씩 저장
for i in range(0, len(token_chunks), 50):
    db.add_documents(token_chunks[i:min(i+50, len(token_chunks))])

Contextual Header를 이용하기 위해, BM25와 Semantic Search를 결합합니다.

In [ ]:
bm25_retriever = BM25Retriever.from_documents(token_chunks, preprocess_func = kiwi_tokenize)
bm25_retriever.k = 5

retriever = db.as_retriever(search_kwargs={"k": 5})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [ ]:
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')